In [40]:
%pip install opencv-python
%pip install huggingface-hub
%pip install diffusers
%pip install segment-anything
%pip install lama-cleaner
%pip install pycocotools

Note: you may need to restart the kernel to use updated packages.


In [51]:
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

!mkdir images
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/groceries.jpg

!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-rqs99ot7
  Running command git clone --filter=blob:none -q https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-rqs99ot7
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
mkdir: cannot create directory ‘images’: File exists
--2023-07-28 13:28:19--  https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 271475 (265K) [image/jpeg]
Saving to: ‘images/truck.jpg.4’

truck.jpg.4         100%[===================>] 265.11K  --.-KB/s    in 0.1s    

2023-07-

In [ ]:
!pip install -q transformers

In [3]:
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
import json
import time

import numpy as np
import matplotlib.pyplot as plt
import cv2

from os import listdir
from os.path import isfile, join

import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

import requests
from PIL import Image

import pycocotools._mask as mask_tool

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import argparse

from transformers import OwlViTProcessor, OwlViTForObjectDetection

PyTorch version: 1.13.1+cu117
Torchvision version: 0.14.1+cu117
CUDA is available: True


In [7]:
photos_dir = 'photos/'
allowed_categories = [ 
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8
]

In [9]:
# read JSON file
def parse_json(file_path):
  jfile = open(file_path)
  file_data = json.load(jfile)
  res = []
  for s in file_data['annotations']:
    img_id = s['image_id']
    category_id = s['category_id']
    s_bbox = s['bbox']
    box = np.array([s_bbox[0], s_bbox[1], s_bbox[0]+s_bbox[2], s_bbox[1]+s_bbox[3]])
    
    res.append({
      "image_id": img_id, 
      "category_id": category_id, 
      "bbox": [int(box[0]), int(box[1]), int(box[2]), int(box[3])], 
      "score": 1
  })

  return res

In [10]:
def restrict_json(file_path):
    jfile = open(file_path)
    file_data = json.load(jfile)
    res = []
    for s in file_data['annotations']:
      img_id = s['image_id']
      category_id = s['category_id']
      s_bbox = s['bbox']
      box = np.array([s_bbox[0], s_bbox[1], s_bbox[0]+s_bbox[2], s_bbox[1]+s_bbox[3]])
      
      res.append({
        "image_id": img_id, 
        "category_id": category_id, 
        "bbox": [int(box[0]), int(box[1]), int(box[2]), int(box[3])], 
        "score": 1
    })

    return res

In [11]:
def detect_bboxes(img_name, texts, img_id, category_id, marked_bbox):
  processor = OwlViTProcessor.from_pretrained("google/owlvit-base-patch32")
  model = OwlViTForObjectDetection.from_pretrained("google/owlvit-base-patch32")

  image = Image.open(photos_dir + img_name)

  inputs = processor(text=texts, images=image, return_tensors="pt")
  outputs = model(**inputs)

  target_sizes = torch.Tensor([image.size[::-1]])
  results = processor.post_process_object_detection(outputs=outputs, target_sizes=target_sizes, threshold=0.1)

  # img = cv2.imread(photos_dir + img_name)
  img = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)
  i = 0  # Retrieve predictions for the first image for the corresponding text queries
  res = []
  text = texts[i]
  font = cv2.FONT_HERSHEY_SIMPLEX
  boxes, scores, labels = results[0]["boxes"], results[0]["scores"], results[0]["labels"]
  print("max score is ", torch.argmax(scores))
  for box, score, label in zip(boxes, scores, labels):
      box = [int(i) for i in box.tolist()]
      print(f"Detected {text[label]} with confidence {round(score.item(), 3)} at location {box}")
      res.append({
      "image_id": img_id, 
      "category_id": category_id, 
      "bbox": [int(box[0]), int(box[1]), int(box[2]), int(box[3])], 
      "score": round(score.item(), 3)
  })
      img = cv2.rectangle(img, [int(box[0]), int(box[1])], [int(box[2]), int(box[3])], (255,0,0), 5)
      img = cv2.rectangle(img, [int(marked_bbox[0]), int(marked_bbox[1])], [int(marked_bbox[2]), int(marked_bbox[3])], (0,0,255), 3)
      if box[3] + 25 > 768:
          y = box[3] - 10
      else:
          y = box[3] + 25 
  
  # fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis
  # ax.imshow(img[:,:,::-1])
  # fig.savefig(f"result_photos/{img_id}_{category_id}.png")   # save the figure to file
  # plt.close(fig)  
  # plt.imshow(img[:,:,::-1])
    
  return res

In [12]:
jfile = open('./dataset/annotation_example.json')
file_data = json.load(jfile)

owlvit_result = []
i=0
for s in file_data['annotations']:
    print(i)
    img_id = s['image_id']
    category_id = s['category_id']
    s_bbox = s['bbox']
    marked_bbox = [s_bbox[0], s_bbox[1], s_bbox[0]+s_bbox[2], s_bbox[1]+s_bbox[3]]
    img_path = file_data['images'][img_id-1]['file_name']
    text = ["a picture of " + file_data['categories'][category_id-1]['name']]
    
    if category_id in allowed_categories:
        print('detect ', img_path, text, category_id)
        detections = detect_bboxes(img_path, [text], img_id, category_id, marked_bbox)
        # print(detections)
        
        owlvit_result.append(detections)
        print(owlvit_result)
        print('--------------------')

    i+=1
    
    
json_string = json.dumps(owlvit_result)
with open('bbox_prediction_result.json', 'w') as f:
    f.write(json_string)  

In [14]:
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))   
    
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

In [30]:
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

SAM model imported


In [ ]:
def convert_mask(mask):
    binary_mask = mask.cpu().numpy().squeeze().astype(np.uint8)

    # Find the contours of the mask
    contours, hierarchy = cv2.findContours(binary_mask,
                                        cv2.RETR_EXTERNAL,
                                        cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour based on area
    largest_contour = max(contours, key=cv2.contourArea)

    # Get the new bounding box
    bbox = [int(x) for x in cv2.boundingRect(largest_contour)]

    # Get the segmentation mask for object 
    segmentation = largest_contour.flatten().tolist()
    
    return segmentation

In [ ]:
def get_annotation(img_id, category_id, segmentation, annotation_data):
    annotation = {
        "image_id": img_id,
        "category_id": category_id,
        "segmentation": segmentation,
        "area": int(cv2.contourArea(contours[0])),
        "bbox": [int(x) for x in cv2.boundingRect(contours[0])],
        "iscrowd": 0,
        "score": score_bbox
    }
    
    photo = annotation_data['images'][img_id-1]
    h, w = photo['height'], photo['width']
    print(annotation_data['images'][img_id-1]['file_name'], h, w)
    segm = annotation['segmentation']
    # print(segm)
    if type(segm) == list:
        # polygon -- a single object might consist of multiple parts
        # we merge all parts into one mask rle code
        rles = mask_tool.frPyObjects([segm], h, w)
        # print(rles[0])
    
    annotation['segmentation'] = {
        "size": rles[0]['size'], 
        "counts": rles[0]['counts'].decode("utf-8")
    }
    
    return annotation

In [ ]:
print("start")
jfile = open('dataset/owl-vit_result2.json')
file_data = json.load(jfile)

jfile = open('dataset/restricted_result.json')
img_data = json.load(jfile)

segm_result = []
i=0
for s in file_data:
    print(i)
    img_id = s['image_id']
    category_id = s['category_id']
    img_name = img_data['images'][img_id-1]['file_name']
    bbox = s['bbox']
    score_bbox = s['score']

    image = cv2.imread(photos_dir + img_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert the bounding box to a tensor
    input_box = torch.tensor(bbox, device=predictor.device)
    transformed_box = predictor.transform.apply_boxes_torch(input_box, image.shape[:2])

    # Set the image
    predictor.set_image(image)

    # Predict the mask
    mask, _, _ = predictor.predict_torch(
        point_coords=None,
        point_labels=None,
        boxes=transformed_box,
        multimask_output=False,
    )   

    binary_mask = mask.cpu().numpy().squeeze().astype(np.uint8)

    # Find the contours of the mask
    contours, hierarchy = cv2.findContours(binary_mask,
                                        cv2.RETR_EXTERNAL,
                                        cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour based on area
    largest_contour = max(contours, key=cv2.contourArea)

    # Get the new bounding box
    bbox = [int(x) for x in cv2.boundingRect(largest_contour)]

    # Get the segmentation mask for object 
    segmentation = largest_contour.flatten().tolist()

    # print(segmentation)
        
    # Define annotation in COCO format
    annotation = {
        "image_id": img_id,
        "category_id": category_id,
        "segmentation": segmentation,
        "area": int(cv2.contourArea(contours[0])),
        "bbox": [int(x) for x in cv2.boundingRect(contours[0])],
        "iscrowd": 0,
        "score": score_bbox
    }
    
    photo = img_data['images'][img_id-1]
    h, w = photo['height'], photo['width']
    print(img_data['images'][img_id-1]['file_name'], h, w)
    segm = annotation['segmentation']
    # print(segm)
    if type(segm) == list:
        # polygon -- a single object might consist of multiple parts
        # we merge all parts into one mask rle code
        rles = mask_tool.frPyObjects([segm], h, w)
        # print(rles[0])
    
    annotation['segmentation'] = {
        "size": rles[0]['size'], 
        "counts": rles[0]['counts'].decode("utf-8")
    }
    
    # print(annotation['segmentation'])
    segm_result.append(annotation)
    # print(annotation)
    i+=1

In [ ]:
json_string = json.dumps(segm_result)

with open('dataset/segm_predicted_result.json', 'w') as f:
    f.write(json_string) 